In [2]:
import os

import cv2
import numpy as np
from dotenv import load_dotenv

# Classification for the base classes

load_dotenv()

# Specify the path to the dataset directory
dataset_dir = os.getenv('DATASET_DIR')

target_size = (128, 128)

In [3]:
# Get all images
def read_images_from_directory(directory):
    images = []
    labels = []

    class_names = []
    for root, dirs, files in os.walk(directory):
        for class_name in dirs:
            class_directory = os.path.join(root, class_name)
            for filename in os.listdir(class_directory):
                file_path = os.path.join(class_directory, filename)
                if os.path.isfile(file_path):
                    image = cv2.imread(file_path, cv2.COLOR_BGR2GRAY)
                    if image is not None:
                        image = cv2.resize(image, target_size)
                        images.append(image)
                        labels.append(os.path.basename(root) + "." + class_name)
                        if class_name not in class_names:
                            class_names.append(class_name)

    images = np.array(images)
    labels = np.array(labels)
    class_names = np.array(class_names)

    return images, labels, class_names

In [4]:
images, labels, class_names = read_images_from_directory(dataset_dir)
images_with_labels = list(zip(images, labels))
print("Number of images:", len(images))
print("Number of labels:", len(labels))
print("Class names:", class_names)
print("Labels:", labels)

Number of images: 22
Number of labels: 22
Class names: ['senior' 'handicaped' 'child' 'adult' 'yellow' 'red' 'green' 'curvy'
 'straight' 'small' 'big' 'agriculture_vehicle' 'bus' 'sportscar' 'danger'
 'information' 'restriction' 'speed_limit' 'yield']
Labels: ['pedestrian.senior' 'pedestrian.handicaped' 'pedestrian.handicaped'
 'pedestrian.child' 'pedestrian.adult' 'trafic_light.yellow'
 'trafic_light.red' 'trafic_light.green' 'lane.curvy' 'lane.straight'
 'animal.small' 'animal.big' 'car.agriculture_vehicle' 'car.bus'
 'car.sportscar' 'traffic_sign.danger' 'traffic_sign.information'
 'traffic_sign.restriction' 'traffic_sign.restriction'
 'traffic_sign.restriction' 'traffic_sign.speed_limit'
 'traffic_sign.yield']


In [5]:
print("First image: ", images_with_labels[0][0])
print("First image - label: ", images_with_labels[0][1])

First image:  [[[ 41  42  38]
  [ 53  53  49]
  [ 80  81  77]
  ...
  [126 130 114]
  [131 123 110]
  [107  98  88]]

 [[ 36  37  33]
  [ 62  64  60]
  [ 86  90  79]
  ...
  [116 123 110]
  [148 139 129]
  [131 124 109]]

 [[ 33  34  30]
  [ 57  58  54]
  [ 63  65  59]
  ...
  [128 132 113]
  [ 89  83  85]
  [145 136 122]]

 ...

 [[108 106  93]
  [103  98  83]
  [ 65  65  51]
  ...
  [132 135 126]
  [129 130 120]
  [140 136 130]]

 [[128 126 112]
  [118 118 104]
  [ 92  91  77]
  ...
  [122 120 112]
  [128 128 116]
  [131 130 116]]

 [[113 112  98]
  [ 81  81  66]
  [120 114 103]
  ...
  [135 135 123]
  [131 135 123]
  [128 128 114]]]
First image - label:  pedestrian.senior


In [6]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Split the dataset into training and testing sets
train_images, test_images, train_labels0, test_labels0 = train_test_split(images, labels, test_size=0.2,
                                                                          random_state=42)

# Convert the labels to one-hot encoded vectors
num_classes = len(np.unique(labels))

print(num_classes)

label_encoder = LabelEncoder()
train_labels1 = label_encoder.fit_transform(train_labels0)
test_labels1 = label_encoder.fit_transform(test_labels0)

from tensorflow.keras.utils import to_categorical

train_labels = to_categorical(train_labels1, num_classes)
test_labels = to_categorical(test_labels1, num_classes)

19


2023-07-05 18:42:54.913806: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-05 18:42:54.966372: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-05 18:42:55.229401: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-05 18:42:55.231298: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-05 18:42:56.149319: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [93]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential()

# Add convolutional and pooling layers
# Fixme
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(target_size[0], target_size[1], 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Add more convolutional and pooling layers if desired

# Flatten the output from the previous layer
model.add(Flatten())

# Add fully connected layers
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

ValueError: Input 0 of layer "conv2d_1" is incompatible with the layer: expected min_ndim=4, found ndim=3. Full shape received: (None, 128, 128)

In [ ]:
num_epochs = 1
batch_size = 10

print(train_images.shape)
print(train_labels.shape)
print(test_images.shape)
print(test_labels.shape)

In [ ]:
# Train the model
model.fit(train_images, train_labels, epochs=num_epochs, batch_size=batch_size,
          validation_data=(test_images, test_labels))

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(test_images, test_labels)
print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')